## Meas/Coords Validation

This goal of this Notebook is to show that real data can be mapped on Measure classes and that the Measure object quantities can be retrieved by generice code.

We proceed with data mapped on individual Measure objects in order to avoid bias possibly introduced by some host model.

This workflow validates the [Measure](https://github.com/ivoa/modelinstanceinvot-code) model (and its coordinates(https://github.com/ivoa-std/CoordinateDM)) in the extand of the the implemented classes.


### Test Case

The VOtable has been queried from tme ESAC archive (https://gea.esac.esa.int/tap-server/tap) with the following query:
```
SELECT TOP 100 gaiadr2.gaia_source.designation , gaiadr2.gaia_source.ra , gaiadr2.gaia_source.ra_error , gaiadr2.gaia_source."dec" , gaiadr2.gaia_source.dec_error , gaiadr2.gaia_source.parallax , gaiadr2.gaia_source.parallax_error , gaiadr2.gaia_source.pmra , gaiadr2.gaia_source.pmra_error , gaiadr2.gaia_source.pmdec , gaiadr2.gaia_source.pmdec_error
 FROM  gaiadr2.gaia_source
 WHERE ( CONTAINS(POINT('ICRS', ra, "dec"), CIRCLE('ICRS', 162.328814, -53.319466, 0.016666666666666666)) = 1 )
```

We select positions, parallax and proper motions around `luhman 16`. 

The test goal is to validate the 3D position. To get that third dimension, we map the Gaia parallax on `Position.dist`. This way to proceed requires to client ot be able to detect this pattern and to transform the parallax into a distance.

There is currently no standard method to handle this sort of implicit transformation. We are doing it here just because we know it is part the test.

This opens the issue of defining a client behaviour when the <FIELD> unit does not match the <ATTRIBUTE> unit.

    
- We are using the annotation [syntax](https://github.com/ivoa-std/ModelInstanceInVot) that has been designed after the 2021 workshop.
- The Python code used for this notebook is being [developped](https://github.com/ivoa/modelinstanceinvot-code) to design qnd validate the processing of model annotation.
- This notebook does not pretend to have any scientific value, it is juste a validation case for the mapping syntax 


### Package Init
- Import packages
- Set the data directory
- Print to check

In [1]:
import sys
import os
import logging
import matplotlib
import mplcursors
import matplotlib.pyplot as plt 
import numpy as np
from astropy.io.votable import parse

base_dir = os.path.realpath(os.path.join(os.getcwd(), ".."))
data_dir = os.path.join(base_dir, "mivot_code", "examples", "data/")

print(os.getcwd())
print(base_dir)
print(data_dir)
sys.path.append(os.path.realpath(base_dir))

from mivot_code.client.xml_interpreter.model_viewer import ModelViewer
from mivot_code.utils.quantity_converter import QuantityConverter


/Users/laurentmichel/Documents/seafile/Seafile/workspaces/git/pyvo_mivot_bench/notebook
/Users/laurentmichel/Documents/seafile/Seafile/workspaces/git/pyvo_mivot_bench
/Users/laurentmichel/Documents/seafile/Seafile/workspaces/git/pyvo_mivot_bench/mivot_code/examples/data/


ModuleNotFoundError: No module named 'mivot_code'

## VOTable parsing
The VOTable is parsed by Astropy as usual

In [2]:
votable = parse(os.path.join(data_dir, "gaia_luhman16.xml"))

FileNotFoundError: [Errno 2] No such file or directory: '/Users/laurentmichel/Documents/seafile/Seafile/workspaces/git/pyvo_mivot_bench/mivot_code/examples/data/gaia_luhman16.xml'

## Building the Model Viewer
The model viewer wrapps the VOTable data iterator to provide model views of the current data raw
The model viewer supports 3 serializations:
- XML
- JSON 
- STC components (used here)

In the future, the resource selection should be handled by the either Astropy or PyVO, but meanwhile this is not implemented, the job is done by hand. 
We assume that the 1st resource is annotated.

In [ ]:
for resource in votable.resources:
    mviewer = ModelViewer(resource, votable_path=os.path.join(data_dir, "gaia_luhman16.xml"))
    break

## Connecting the Data table we Want to Browse
In this example we just have one table that is annotated. 
- We assume we will work on data of this table
- We do not look for complex objects that could be located in some other tables or in the GLOBALS block 

In [ ]:
mviewer.connect_table('Results')


## Using Data Model Views
The model viewer iterate over the data row by using the Astropy parser. The last read row is stored internally to get available to be processed by the viewer.

In the current the example, we know that each data rows contains one 3D position and one proper motion.
The viewer provides tools to discover these quantities, but this is not the purpose of the exercise. 

For each row, we extract Measures instances of those quantities and we store them in Python lists ready to be displayed by matplotlib.
The parallax are transformed as distances and the proper motion is given with tooltips.


In [ ]:
times = []
ras = []
decs = []
dist = []       
must_convert = False
first_row = True
while True:
    row = mviewer.get_next_row() 
    if row is None:
        break  
            
    position = mviewer.get_stc_positions()[0]

    if first_row is True:
        first_row = False
        if position.coord.dist.unit == "parsec":
            must_convert = True
                    
    if must_convert is True:       
        position.coord.dist.value = QuantityConverter.parallax_to_distance(position.coord.dist.value)
        position.error.plus[2].value = QuantityConverter.parallax_to_distance(position.error.plus[2].value)
        position.error.minus[2].value = QuantityConverter.parallax_to_distance(position.error.minus[2].value)
            
    if not np.isnan(position.coord.dist.value) and position.coord.dist.value < 5000.0:
        ras.append(position.coord.lon.value)    
        decs.append(position.coord.lat.value)   
        dist.append(position.coord.dist.value)   
        pm = mviewer.get_stc_measures_by_ucd("pos.pm")[0]
        times.append(f"Proper Motion ({pm.coord.lon.unit}):\nra:{pm.coord.lon.value:.2f} \ndec:{pm.coord.lat.value:.2f}")


## Matplotlib setup

Activate dynamic plots to make the tooltips working.

In [ ]:
%matplotlib widget

## Lets plot

In [ ]:
matplotlib.font_manager: logging.getLogger('matplotlib.font_manager').setLevel(logging.WARNING)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(ras, decs, dist)
ax.set_xlabel('RA')
ax.set_ylabel('DEC')
ax.set_zlabel('Dist (parsec)')
mplcursors.cursor(hover=True).connect("add", lambda sel: sel.annotation.set_text(times[sel.index]))
plt.show()